In [ ]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.feature_selection import RFE
from sklearn.ensemble import  RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import GroupShuffleSplit, GridSearchCV, GroupKFold
from sklearn.pipeline import Pipeline
from sklearn.neural_network import MLPRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import DotProduct, RBF, ConstantKernel as RationalQuadratic
from sklearn.dummy import DummyRegressor

from scipy.stats import sem, t
from skopt import BayesSearchCV

from xgboost import XGBRegressor
import shap
from tqdm import tqdm

import warnings
warnings.filterwarnings("ignore")

In [ ]:
### LOAD ALL DATAFRAMES (JUST MMSE USED HERE FOR EXAMPLE) ###

mmse = pd.read_csv()

mmse_standard = mmse.copy() # Used below for baseline model evaluation

mmse['MMSE_ROC_12'] = mmse['MMSE_ROC_12'] + mmse['Baseline MMSE']

# Model Evaluation

In [ ]:
### DEFINE A FUNCTION TO MAINTAIN SEX DISTRIBUTION ACROSS TRAIN TEST SPLITS ###

def StratifiedGroupShuffleSplit(X, y, groups, n_splits, test_size=0.2, random_state=42):
    # Split data by sex
    f_idx = X[X['Sex'] == 2].index
    X_f, y_f, groups_f = X.loc[f_idx], y.loc[f_idx], groups.loc[f_idx]

    m_idx = X[X['Sex'] == 1].index
    X_m, y_m, groups_m = X.loc[m_idx], y.loc[m_idx], groups.loc[m_idx]

    splits = []

    gss_f = GroupShuffleSplit(n_splits=n_splits, test_size=test_size, random_state=random_state)
    gss_m = GroupShuffleSplit(n_splits=n_splits, test_size=test_size, random_state=random_state)

    for (train_idx_f, test_idx_f), (train_idx_m, test_idx_m) in zip(gss_f.split(X_f, y_f, groups_f), gss_m.split(X_m, y_m, groups_m)):
        X_train_f, X_test_f = X_f.iloc[train_idx_f], X_f.iloc[test_idx_f]
        y_train_f, y_test_f = y_f.iloc[train_idx_f], y_f.iloc[test_idx_f]
        groups_train_f = groups_f.iloc[train_idx_f]

        X_train_m, X_test_m = X_m.iloc[train_idx_m], X_m.iloc[test_idx_m]
        y_train_m, y_test_m = y_m.iloc[train_idx_m], y_m.iloc[test_idx_m]
        groups_train_m = groups_m.iloc[train_idx_m]

        X_train = pd.concat([X_train_f, X_train_m])
        y_train = pd.concat([y_train_f, y_train_m])
        groups_train = pd.concat([groups_train_f, groups_train_m])

        X_test = pd.concat([X_test_f, X_test_m])
        y_test = pd.concat([y_test_f, y_test_m])

        splits.append((X_train, y_train, groups_train, X_test, y_test))

    return splits

In [ ]:
### FUNCTION TO EVALUATE MODEL PERFORMANCE ###

def train_and_evaluate(X_train, y_train, X_test, y_test, groups_train, models=None, search_spaces=None):
    if models is None:
        models = {
            "Ridge": Ridge(max_iter=10000),
            "Lasso": Lasso(max_iter=10000),
            "ElasticNet": ElasticNet(max_iter=10000),
            "XGBoost": XGBRegressor(),
            "RandomForest": RandomForestRegressor(random_state=42)
        }
    
    if search_spaces is None:
        search_spaces = {
            "Ridge": {
                "model__alpha": (1e-3, 1e4, "log-uniform")
            },
            "Lasso": {
                "model__alpha": (1e-3, 10.0, "log-uniform"),
                "model__max_iter": (1000, 10000)
            },
            "ElasticNet": {
                "model__alpha": (1e-3, 10.0, "log-uniform"),
                "model__l1_ratio": (0.1, 0.9, "uniform")
            },
            "XGBoost": {
                "model__n_estimators": (50, 200),
                "model__max_depth": (2, 6),
                "model__learning_rate": (0.01, 0.1, "log-uniform"),
                "model__min_child_weight": (5, 15),  
                "model__subsample": (0.6, 1.0, "uniform"), 
                "model__colsample_bytree": (0.6, 1.0, "uniform"),  
                "model__gamma": (0.1, 5, "log-uniform"),  
                "model__reg_alpha": (0.1, 5.0, "log-uniform"),  
                "model__reg_lambda": (1.0, 10.0, "log-uniform") 
            },
            "RandomForest": {
                "model__n_estimators": (50, 1000),
                "model__max_depth": (3, 20),
            }
        }

    results = {name: {"MSE": [], "MAE": [], "R2": []} for name in models} 

    gkf = GroupKFold(n_splits=5)

    for name, model in models.items():
        pipeline = Pipeline([
            ('scaler', StandardScaler()),
            ('rfe', RFE(model)),
            ('model', model)
        ])
        
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore")
            bayes_search = BayesSearchCV(
                estimator=pipeline,
                search_spaces=search_spaces[name],
                cv=gkf, 
                n_iter=25,
                scoring="neg_mean_squared_error",  
                random_state=42,
                n_jobs=-1
            )

            bayes_search.fit(X_train, y_train, groups=groups_train)

        best_model = bayes_search.best_estimator_
        y_pred = best_model.predict(X_test)

        mse = mean_squared_error(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test,y_pred)

        results[name]["MSE"].append(mse)
        results[name]["MAE"].append(mae)
        results[name]["R2"].append(r2)

    return results

In [ ]:
### NESTED CROSS VALIDATION: IMPLEMENTS train_and_evaluate() OVER TEN SPLITS ###

def nested_cv(X, y, groups):
    all_results = []
    summary_rows = []

    splits = StratifiedGroupShuffleSplit(X, y, groups, n_splits=10)

    for X_train, y_train, groups_train, X_test, y_test in tqdm(splits, total=10, desc="Outer CV (Splits)"):
        split_results = train_and_evaluate(X_train, y_train, X_test, y_test, groups_train)
        all_results.append(split_results)

    for name in all_results[0].keys():
        mse_values = []
        mae_values = []
        r2_values = []

        for split in all_results:
            mse_values.extend(split[name]["MSE"])
            mae_values.extend(split[name]["MAE"])
            r2_values.extend(split[name]["R2"])

        avg_mse = sum(mse_values) / len(mse_values)
        avg_mae = sum(mae_values) / len(mae_values)
        avg_r2 = sum(r2_values) / len(r2_values)

        mse_sem = sem(mse_values)
        mae_sem = sem(mae_values)
        r2_sem = sem(r2_values)

        mse_ci = 1.96 * mse_sem
        mae_ci = 1.96 * mae_sem
        r2_ci = 1.96 * r2_sem

        summary_rows.append({
            "Model": name,
            "MSE (95% CI)": f"{avg_mse:.1f} ({avg_mse - mse_ci:.1f} – {avg_mse + mse_ci:.1f})",
            "MAE (95% CI)": f"{avg_mae:.1f} ({avg_mae - mae_ci:.1f} – {avg_mae + mae_ci:.1f})",
            "R^2 (95% CI)": f"{avg_r2:.1f} ({avg_r2 - r2_ci:.1f} – {avg_r2 + r2_ci:.1f})",
        })

    summary_df = pd.DataFrame(summary_rows).set_index("Model")
    print("\nSummary of Results:")
    # print(summary_df)
    print(summary_df.to_latex())

    return summary_df

In [ ]:
### FOR MLP and GP ###

def train_and_evaluate_mlpgp(X_train, y_train, X_test, y_test, groups_train, models=None, search_spaces=None):
    input_neurons = X_train.shape[1]
    HL_1 = input_neurons
    HL_2 = int(HL_1 * 0.75)
    HL_3 = int(HL_2 * 0.75)
    
    if models is None:
        models = {
            "MLP": MLPRegressor(max_iter=1000, hidden_layer_sizes=(HL_1, HL_2, HL_3), random_state=42),
            "GP": GaussianProcessRegressor(random_state=42),
        }
    
    if search_spaces is None:
        search_spaces = {
            "MLP": {
                "model__activation": ['identity', 'logistic', 'tanh', 'relu'],
                "model__solver": ['lbfgs', 'sgd', 'adam']
            },
            "GP": {
                "model__alpha": [0.0001, 0.001, 0.01, 0.1, 1, 10],
                "model__kernel": (
                    [RBF(length_scale) for length_scale in np.logspace(-3, 1, 5)] +
                    [DotProduct(sigma_0) for sigma_0 in np.logspace(-3, 1, 5)] +
                    [RationalQuadratic(length_scale=l, alpha=a)
                    for l in np.logspace(-3, 1, 3)
                    for a in np.logspace(-3, 1, 3)]
                )
            }
        }

    results = {name: {"MSE": [], "MAE": [], "R2": []} for name in models} 

    gkf = GroupKFold(n_splits=5)

    for name, model in models.items():
        # print(f"Optimizing {name}...")
        pipeline = Pipeline([
            ('scaler', StandardScaler()),
            ('model', model)
        ])
        
        search = GridSearchCV(
            estimator=pipeline,
            param_grid=search_spaces[name],
            scoring="neg_mean_squared_error",
            cv=gkf,
        )

        search.fit(X_train, y_train, groups=groups_train)

        best_model = search.best_estimator_
        y_pred = best_model.predict(X_test)

        mse = mean_squared_error(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test,y_pred)

        results[name]["MSE"].append(mse)
        results[name]["MAE"].append(mae)
        results[name]["R2"].append(r2)

    return results

In [ ]:
def nested_cv_mlpgp(X, y, groups):   
    all_results = []
    summary_rows = []

    splits = StratifiedGroupShuffleSplit(X, y, groups, n_splits=10)

    for X_train, y_train, groups_train, X_test, y_test in tqdm(splits, total=10, desc="Outer CV (Splits)"):
        split_results = train_and_evaluate_mlpgp(X_train, y_train, X_test, y_test, groups_train)
        all_results.append(split_results)

    for name in all_results[0].keys():  # Loop over model names
        mse_values = []
        mae_values = []
        r2_values = []

        for split in all_results:
            mse_values.extend(split[name]["MSE"])
            mae_values.extend(split[name]["MAE"])
            r2_values.extend(split[name]["R2"])

        avg_mse = sum(mse_values) / len(mse_values)
        avg_mae = sum(mae_values) / len(mae_values)
        avg_r2 = sum(r2_values) / len(r2_values)

        mse_sem = sem(mse_values)
        mae_sem = sem(mae_values)
        r2_sem = sem(r2_values)

        mse_ci = 1.96 * mse_sem
        mae_ci = 1.96 * mae_sem
        r2_ci = 1.96 * r2_sem

        summary_rows.append({
            "Model": name,
            "MSE (95% CI)": f"{avg_mse:.1f} ({avg_mse - mse_ci:.1f} – {avg_mse + mse_ci:.1f})",
            "MAE (95% CI)": f"{avg_mae:.1f} ({avg_mae - mae_ci:.1f} – {avg_mae + mae_ci:.1f})",
            "R^2 (95% CI)": f"{avg_r2:.1f} ({avg_r2 - r2_ci:.1f} – {avg_r2 + r2_ci:.1f})",
        })

    summary_df = pd.DataFrame(summary_rows).set_index("Model")
    print("\nSummary of Results:")
    print(summary_df.to_latex())

    return summary_df

In [ ]:
X_mmse = mmse.drop(columns=['MMSE_ROC_12', 'Dyad'])
y_mmse = mmse['MMSE_ROC_12']
groups_mmse = mmse['Dyad']

# Repeat for all other feature combinations

In [ ]:
nested_cv(X_mmse, y_mmse, groups_mmse)

In [ ]:
nested_cv_mlpgp(X_mmse, y_mmse, groups_mmse)

# Baseline Model Training and Evaluation

In [ ]:
mean_mmse_roc = mmse_standard['MMSE_ROC_12'].mean()
mmse_standard['MMSE_ROC_12'] = mmse_standard['Baseline MMSE'] + mean_mmse_roc

X_mmse_standard = mmse_standard.drop(columns=['MMSE_ROC_12', 'Dyad'])
y_mmse_standard = mmse_standard['MMSE_ROC_12']
groups_mmse_standard = mmse_standard['Dyad']

In [ ]:
splits = StratifiedGroupShuffleSplit(X_mmse_standard, y_mmse_standard, groups_mmse_standard, n_splits=10)
naive_model = DummyRegressor(strategy='mean')
scaler = StandardScaler()

mse_scores = []
mae_scores = []
r2_scores = []

for X_train, y_train, groups_train, X_test, y_test in tqdm(splits, total=10, desc="Outer CV (Splits)"):
    X_train, X_test = scaler.fit_transform(X_train), scaler.transform(X_test)
    naive_model.fit(X_train, y_train)
    y_pred = naive_model.predict(X_test)

    mse_scores.append(mean_squared_error(y_test, y_pred))
    mae_scores.append(mean_absolute_error(y_test, y_pred))
    r2_scores.append(r2_score(y_test, y_pred))

mse_scores = np.array(mse_scores)
mae_scores = np.array(mae_scores)
r2_scores = np.array(r2_scores)

avg_mse = sum(mse_scores) / len(mse_scores)
avg_mae = sum(mae_scores) / len(mae_scores)
avg_r2 = sum(r2_scores) / len(r2_scores)

mse_sem = sem(mse_scores)
mae_sem = sem(mae_scores)
r2_sem = sem(r2_scores)

mse_ci = 1.96 * mse_sem
mae_ci = 1.96 * mae_sem
r2_ci = 1.96 * r2_sem

print("MSE (95% CI):", f"{avg_mse:.1f} ({avg_mse - mse_ci:.1f} – {avg_mse + mse_ci:.1f})")
print("MAE (95% CI):", f"{avg_mae:.1f} ({avg_mae - mae_ci:.1f} – {avg_mae + mae_ci:.1f})")
print("R^2 (95% CI):", f"{avg_r2:.1f} ({avg_r2 - r2_ci:.1f} – {avg_r2 + r2_ci:.1f})")

# Final model training - on the entire dataset

In [ ]:
def final_training(X, y, groups, models=None, search_spaces=None, models_to_run=None):
    
    if models is None:
        models = {
            "Ridge": Ridge(),
            "Lasso": Lasso(),
            "ElasticNet": ElasticNet(),
            "XGBoost": XGBRegressor(),
            "Random Forest": RandomForestRegressor(random_state=42)
        }
    
    if search_spaces is None:
        search_spaces = {
            "Ridge": {
                "model__alpha": (1e-2, 1e4, "log-uniform")
            },
            "Lasso": {
                "model__alpha": (1e-2, 10.0, "log-uniform"),
                "model__max_iter": (1000, 5000, 10000)
            },
            "ElasticNet": {
                "model__alpha": (1e-2, 10.0, "log-uniform"),
                "model__l1_ratio": (0.1, 0.6, "uniform")
            },
            "XGBoost": {
                "model__n_estimators": (50, 200),
                "model__max_depth": (2, 6),
                "model__learning_rate": (0.01, 0.1, "log-uniform"),
                "model__min_child_weight": (5, 15),  # Less aggressive fitting
                "model__subsample": (0.6, 1.0, "uniform"),  # Control overfitting
                "model__colsample_bytree": (0.6, 1.0, "uniform"),  # Feature selection
                "model__gamma": (0.1, 5, "log-uniform"),  # Prevent unnecessary splits
                "model__reg_alpha": (0.1, 5.0, "log-uniform"),  # L1 regularization
                "model__reg_lambda": (1.0, 10.0, "log-uniform")  # L2 regularization
            },
            "Random Forest": {
                "model__n_estimators": (50, 500),
                "model__max_depth": (3, 10),
            },
        }
    
    gkf = GroupKFold(n_splits=10)

    if models_to_run is not None:
        models = {name: model for name, model in models.items() if name in models_to_run}
        search_spaces = {name: space for name, space in search_spaces.items() if name in models_to_run}

    for name, model in models.items():
        pipeline = Pipeline([
            ('scaler', StandardScaler()),
            ('rfe', RFE(estimator=model)),
            ('model', model)
        ])
        
        # Initialize BayesSearchCV
        bayes_search = BayesSearchCV(
            estimator=pipeline,
            search_spaces=search_spaces[name],
            cv=gkf, 
            n_iter=50,  # Number of parameter combinations to try
            scoring="neg_mean_squared_error",  # Optimize for MSE
            random_state=42,
            n_jobs=-1
        )

        # Fit BayesSearchCV on the training data
        bayes_search.fit(X, y, groups=groups)

        print(f"Best parameters for {name}: {bayes_search.best_params_}")

In [ ]:
final_training(X_mmse, y_mmse, groups_mmse, models_to_run=['ElasticNet'])

# SHAPs

In [ ]:
def train_model(model, X, y):
    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('rfe', RFE(model)),
        ('model', model)
    ])

    pipeline.fit(X, y)

    selected_features_mask = pipeline.named_steps['rfe'].support_
    selected_features = X.columns[selected_features_mask]
    X_selected = X[selected_features]

    model = pipeline.named_steps['model']

    scaler = pipeline.named_steps['scaler']
    X_selected = pd.DataFrame(scaler.fit_transform(X_selected), columns=X_selected.columns)
    
    explainer = shap.Explainer(model, X_selected)
    shap_values = explainer.shap_values(X_selected)

    return shap_values, X_selected

def shap_plot(shap_values, X_selected):

    plt.figure()
    shap.summary_plot(
        shap_values, X_selected, max_display=10, plot_size=(12,5), show=False, plot_type='violin' #, cmap='crest'
    )

    fig, ax = plt.gcf(), plt.gca()
    ax.tick_params(axis='y', labelsize=12)

    for text in ax.texts:
        text.set_color('black')
    for label in ax.get_yticklabels():
        label.set_color('black')
    for label in ax.get_xticklabels():
        label.set_color('black')

    ax.xaxis.label.set_color('black')
    ax.yaxis.label.set_color('black')
    ax.xaxis.grid(False)
    ax.set_xlabel("SHAP value", fontsize=14, color='black')
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_color('black')
    ax.spines['left'].set_linewidth(2)
    ax.spines['bottom'].set_color('black')
    ax.spines['bottom'].set_linewidth(2)

    for line in ax.lines:
        line.set_linewidth(2) 

    ax.set_aspect('auto', adjustable='box')

    plt.show()


In [ ]:
model = ElasticNet(alpha=, l1_ratio=)
shap_values, X_selected=train_model(model, X_mmse, y_mmse)
shap_plot(shap_values, X_selected)